In [31]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
import pandas as pd
import time
import random

In [32]:
def AllCategory():
    wait = WebDriverWait(driver, 10)
    ShopALL = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/section[1]/header/div[2]/div/div/ul/li[1]/a/figcaption/span")))  
    ShopALL.click()
    driver.implicitly_wait(10)
    initial_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == initial_height:
            break
        initial_height = new_height
    time.sleep(30)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')   
    div = soup.find_all('div', class_='CategoryPage-CategoryList')
    Category=[]
    for i in div:
            anchor_tags = i.find_all('a',class_='CategoryPod-Pod')
            for anchor_tag in anchor_tags:
                href_value = anchor_tag.get('href')
                Category.append(href_value)
    if '/store' in Category:
        Category.remove('/store')
    return Category

In [33]:
def Product_link(URL):
    driver.get(URL)
    initial_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == initial_height:
            break
        initial_height = new_height
    time.sleep(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')  
    div = soup.find_all('div', class_='CategoryProductList CategoryProductList CategoryProductList_layout_grid')
    Pro_links=[]
    for i in div:
            anchor_tags = i.find_all('a')
            for anchor_tag in anchor_tags:
                href_value = anchor_tag.get('href')
                Pro_links.append(href_value)
    return Pro_links

In [34]:
def unique(Orignal_list):
    unique_list = [item for index, item in enumerate(Orignal_list) if item not in Orignal_list[:index]]
    return unique_list

In [62]:
def get_product_info(URL):
    driver.get(URL)
    time.sleep(3)
    driver.implicitly_wait(10)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    try:
        P_name_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/section[1]/div/div[2]/div[1]/h1')
                                                    
    except NoSuchElementException as e:
        driver.refresh()
        P_name_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/section[1]/div/div[2]/div[1]/h1')
               
    try:
        SKU_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/section[1]/div/div[2]/article/section[1]/div/span')
                                                    
    except NoSuchElementException as e:
        SKU_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[1]/div/div[2]/article/section[1]/div/span')
                                                    
    try:
        Price_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/section[1]/div/div[2]/article/div[2]/div/div[1]/div/div/div/span/span')
        Sub_product_element = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div[2]/main/section[1]/div/div[2]/article/section[2]/div')
    except NoSuchElementException as e:
        Sub_product_element = soup.find_all('div', class_='GroupedProductsItem-Title')
        if not Sub_product_element:
            Sub_product_element = None
    try:
        Review_count_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/section[1]/div/div[2]/article/div[1]/button')
    except Exception as e:
        Review_count_element = None
    
    try:
        Rating_element = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/main/section[1]/div/div[2]/article/div[1]/div/span')
    except Exception as e:
        Rating_element = None
    df =[]
    if isinstance(Sub_product_element, WebElement) or Sub_product_element is None:
        SKU = SKU_element.text.strip()
        Sub = Sub_product_element.text.strip() if Sub_product_element else None
        product_name = P_name_element.text.strip() 
        Price = Price_element.text.strip()
        Rating = Rating_element.text.strip() if Rating_element else '0'
        ReviewCount= Review_count_element.text.strip() if Review_count_element else '(0 reviews)'
        row = {
                    'PageURL': URL,
                    'SKU': SKU,
                    'ProductName': product_name,
                    'SubProduct' : Sub,
                    'Price':Price,
                    'ReviewCount': ReviewCount,
                    'OverAllRating': Rating
                }
        df.append(row)
    else:
        SKU = SKU_element.text.strip()
        product_name = P_name_element.text.strip() 
        Sub = [element.p.text for element in Sub_product_element]
        Price = [element.span.text for element in Sub_product_element]
        Rating = Rating_element.text.strip() if Rating_element else '0'
        ReviewCount= Review_count_element.text.strip() if Review_count_element else '(0 reviews)'

        for i in range(len(Sub_product_element)):
            row = {
                'PageURL': URL,
                'SKU': SKU,
                'ProductName': product_name,
                'SubProduct' : Sub[i],
                'Price':Price[i],
                'ReviewCount': ReviewCount,
                'OverAllRating': Rating
            }
            df.append(row)
    return df

In [65]:
Web_URL = 'https://www.rareseeds.com'
allowed_user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Safari/14.1.2",
    "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)"   
]
ProPage_links = []
Product_details=[]

In [21]:
user_agents = random.choice(allowed_user_agents)
headers = {'User-Agent': user_agents}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f"user-agent={headers['User-Agent']}")
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
driver.get(Web_URL)
Category = AllCategory()
for i in Category:
    ProPage_links.extend(Product_link(Web_URL + i))
print(len(ProPage_links))
Unique_Links = unique(ProPage_links)
print(len(Unique_Links))
df = pd.DataFrame(Unique_Links)
df.to_csv('Links.csv',index=False)
for i in Unique_Links:
    URL = Web_URL + i
    df = get_product_info(URL)
    print(df)
    Product_details.extend(df)
daf = pd.DataFrame(Product_details)
daf.to_csv('RareSeeds.csv',index=False)
driver.quit()    